<a href="https://colab.research.google.com/github/cyberust/SynergyAnalysis/blob/main/Synergy_Analysis_VC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

# 必要なライブラリのインストール
!pip install PyPDF2 python-docx spacy wordcloud matplotlib seaborn scikit-learn nltk

# spaCyの英語モデルをダウンロード (初回のみ必要)
!python -m spacy download en_core_web_sm

print("ステップ1完了: Driveのマウントとライブラリのインストールが完了しました。")

In [ ]:
import os
import PyPDF2
from docx import Document as DocxDocument # ライブラリ名と区別するため別名
import spacy
from collections import Counter
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords

# NLTKのストップワードリストをダウンロード (初回のみ必要)
try:
    stopwords.words('english')
except LookupError:
    nltk.download('stopwords')
    nltk.download('punkt') # トークナイズに必要

# spaCyの英語モデルをロード
nlp = spacy.load('en_core_web_sm')

# 分析対象の人物リストとファイルパス
# !!! ご自身のファイル構成に合わせてパスを修正してください !!!
BASE_PATH = '/content/drive/MyDrive/' # Google DriveのCVファイルがあるフォルダパス

# 人物名とファイル名のマッピング (実際のファイル名に合わせてください)
# ファイル拡張子はテキスト抽出時に判断するため、ここでは不要
# ただし、ファイル名にピリオドが多い場合は拡張子まで含めて正確に記載推奨
# キーが人物名、バリューがファイル名（拡張子なし or あり）
# 例: 'Arale Cohen': 'Arale_Cohen_CV.pdf'
# 例: 'Yanay Geva': 'Yanay_Geva_Profile' (拡張子が.txtや.docxなどでもOK)
persons_files = {
    'Arale Cohen': 'arale_cohen_profile.txt',    # 例: PDFファイル
    'Yanay Geva': 'yanay_geva_profile.txt',   # 例: DOCXファイル
    'Yasuyuki Sakane': 'yasuyuki_sakane_profile.txt' # 例: TXTファイル
}

# 結果を格納する辞書
analysis_results = {}

print("ステップ2完了: ライブラリのインポートと基本設定が完了しました。")

In [ ]:
import os

def extract_text_from_txt(txt_file_path):
    text = ""
    try:
        with open(txt_file_path, 'r', encoding='utf-8') as file:
            text = file.read()
    except Exception as e:
        print(f"Error reading TXT {txt_file_path}: {e}")
    return text

def get_text_from_file(file_path):
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        return ""

    _, file_extension = os.path.splitext(file_path)
    file_extension_lower = file_extension.lower()

    if file_extension_lower == '.txt':
        return extract_text_from_txt(file_path)
    else:
        print(f"Unsupported file type: {file_extension_lower} for file {file_path}. This version only supports .txt files.")
        return ""

# テキスト抽出の実行
# !!! BASE_PATH と persons_files はステップ2で定義されている前提 !!!
# 例:
# BASE_PATH = '/content/drive/MyDrive/VC_Analysis_Data/'
# persons_files = {
#     'Arale Cohen': 'arale_cohen_profile.txt',
#     'Yanay Geva': 'yanay_geva_profile.txt',
#     'Yasuyuki Sakane': 'yasuyuki_sakane_profile.txt'
# }

raw_texts = {}
for person, file_name in persons_files.items():
    full_path = os.path.join(BASE_PATH, file_name)
    print(f"Processing {person}'s CV: {full_path}")
    raw_texts[person] = get_text_from_file(full_path)
    if not raw_texts[person]:
        print(f"Could not extract text for {person}.")
    else:
        print(f"Successfully extracted text for {person} (length: {len(raw_texts[person])} chars).")

analysis_results = {} # analysis_resultsが未定義の場合に備えて初期化
analysis_results['raw_texts'] = raw_texts
print("\nステップ3完了: ファイルからのテキスト抽出が試みられました。")

In [ ]:
stop_words_english = set(stopwords.words('english'))
custom_stopwords_to_add = [
    "business", "market", "strategic", "japanese", "japan", "israeli", "development",
    "investment", "line", "provide", "information", "target", "individual",
    "experience", "year", "leadership", "company", "team", "project", "skill",
    "ability", "knowledge", "strong", "also", "work", "role", "new", "growth",
    "strategy", "management", "solution", "customer", "value", "global", "organization"
    # その他、出力を見ながら不要と思われる単語を追加
]
if 'stop_words_english' in globals():
    stop_words_english.update(custom_stopwords_to_add)
else:
    from nltk.corpus import stopwords
    stop_words_english = set(stopwords.words('english'))
    stop_words_english.update(custom_stopwords_to_add)

print(f"Updated stop_words_english. New count: {len(stop_words_english)}")

import re # ステップ2でインポート済みのはず
import spacy # ステップ2でインポート済みのはず
# nlp = spacy.load('en_core_web_sm') # ステップ2でロード済みのはず

def preprocess_text(text_original):
    if not text_original:
        return ""

    text = text_original.lower()
    text = re.sub(r'\S*@\S*\s?', '', text)  # メールアドレス
    text = re.sub(r'http\S+|www\S+', '', text) # URL
    # text = re.sub(r'\b\d+\b', '', text) # 数字のみの単語除去はtoken.is_digitで対応するのでコメントアウト
    text = re.sub(r'[^a-zA-Z0-9\s.-]', '', text) # アルファベット(大小)、数字、空白、ドット、ハイフンを残す

    doc = nlp(text) # グローバルスコープのnlpモデルを使用
    lemmatized_tokens = []
    for token in doc:
        # 条件1: ストップワードでない
        # 条件2: 句読点でない
        # 条件3: 空白でない
        # 条件4: 純粋な数字でない (token.is_digit)
        # 条件5: レンマの長さが1より大きい (例: 2文字以上の単語を残す。必要に応じて >0 や >2 に変更)
        # 条件6: レンマが空白文字のみでない
        if not token.is_stop and \
           not token.is_punct and \
           not token.is_space and \
           not token.is_digit and \
           len(token.lemma_) > 1 and \
           not token.lemma_.isspace():
            lemmatized_tokens.append(token.lemma_)

    # デバッグ用に、処理後のトークンの一部を表示
    # print(f"--- Preprocessing Debug for preprocess_text ---")
    # print(f"Original text (first 50): {text_original[:50]}")
    # print(f"Text after regex (first 50): {text[:50]}")
    # print(f"Lemmatized tokens (first 10): {lemmatized_tokens[:10]}")

    return " ".join(lemmatized_tokens)

# 前処理の実行
processed_texts = {}
for person, text in raw_texts.items():
    if text:
        processed_texts[person] = preprocess_text(text)
        print(f"Processed text for {person} (length: {len(processed_texts[person])} chars).")
        # 前処理後のテキストの冒頭部分を表示 (確認用)
        # print(f"Preview for {person} (processed):\n{processed_texts[person][:200]}...\n")
    else:
        processed_texts[person] = ""
        print(f"Skipping processing for {person} due to empty raw text.")


analysis_results['processed_texts'] = processed_texts
print("\nステップ4完了: テキスト前処理が完了しました。")

In [ ]:
from collections import Counter # これはステップ2でインポート済みのはずですが、念のため
import matplotlib.pyplot as plt # ステップ2でインポート済みのはず
import seaborn as sns # ステップ2でインポート済みのはず
import pandas as pd # pandasをインポート
import re # ステップ2でインポート済みのはず
# spaCyやnltk関連もステップ2,4でロード・ダウンロード済みのはず

# !!! スキルキーワードリスト (ご自身の関心に合わせて拡張・修正してください) !!!
# 技術スキル、ビジネススキル、ソフトスキルなど
SKILL_KEYWORDS = [
    'python', 'java', 'c++', 'javascript', 'react', 'angular', 'vue', 'node.js',
    'machine learning', 'deep learning', 'artificial intelligence', 'ai', 'nlp', 'natural language processing',
    'data science', 'data analysis', 'big data', 'hadoop', 'spark',
    'cloud computing', 'aws', 'azure', 'gcp',
    'devops', 'docker', 'kubernetes',
    'agile', 'scrum',
    'project management', 'product management',
    'business development', 'strategy', 'strategic planning', 'market analysis', 'financial modeling',
    'vc', 'venture capital', 'private equity', 'investment', 'due diligence', 'fundraising',
    'leadership', 'team management', 'communication', 'negotiation',
    'blockchain', 'fintech', 'saas', 'b2b', 'b2c', 'iot',
    'global expansion', 'market entry', # 日本市場関連のキーワード
    # Yasuyuki Sakane氏の専門性に合わせて追加
    'computer vision', 'robotics', 'cybersecurity', 'quantum computing',
    # Arale Cohen氏、Yanay Geva氏の専門性に合わせて追加
    'deal sourcing', 'portfolio management', 'exit strategy', 'term sheet'
]

def extract_skills(text, skill_keywords):
    if not text:
        return []

    found_skills = set()
    text_lower = text.lower()

    for skill in skill_keywords:
        pattern = r'\b' + re.escape(skill.lower()) + r'\b'
        if re.search(pattern, text_lower):
            found_skills.add(skill)

    return sorted(list(found_skills))

# スキル抽出の実行
# !!! processed_texts はステップ4で定義されている前提 !!!
# 例:
# processed_texts = {
#     'Arale Cohen': "processed text for arale...",
#     'Yanay Geva': "processed text for yanay...",
#     'Yasuyuki Sakane': "processed text for yasuyuki..."
# }
# analysis_results = {'processed_texts': processed_texts} # analysis_results が存在し、processed_textsを含むこと

extracted_skills = {}
# processed_texts が analysis_results 内に存在するか確認
if 'processed_texts' in analysis_results and isinstance(analysis_results['processed_texts'], dict):
    current_processed_texts = analysis_results['processed_texts']
else:
    # analysis_resultsにprocessed_textsがない、または適切な型でない場合のフォールバック
    # この場合、processed_textsがグローバルスコープに存在することを期待
    if 'processed_texts' not in globals() or not isinstance(processed_texts, dict):
        print("Error: 'processed_texts' is not defined or not a dictionary. Please ensure Step 4 has been run successfully.")
        current_processed_texts = {} # エラーを防ぐため空の辞書を割り当て
    else:
        current_processed_texts = processed_texts


for person, text in current_processed_texts.items():
    if text:
        extracted_skills[person] = extract_skills(text, SKILL_KEYWORDS)
        print(f"Extracted skills for {person}: {extracted_skills[person]}")
    else:
        extracted_skills[person] = []
        print(f"Skipping skill extraction for {person} due to empty processed text.")

# analysis_results に extracted_skills を格納
if 'analysis_results' not in globals() or not isinstance(analysis_results, dict):
    analysis_results = {} # analysis_resultsが未定義の場合、初期化
analysis_results['extracted_skills'] = extracted_skills


# スキルセットの比較と可視化
all_skills_list = []
for person_skills in extracted_skills.values():
    all_skills_list.extend(person_skills)

if not all_skills_list: # all_skills_listが空の場合の処理
    print("No skills were extracted for any person. Skipping skill visualization.")
else:
    skill_counts = Counter(all_skills_list)

    # 全員のスキル保有状況を棒グラフで表示
    plt.figure(figsize=(15, 8))
    # skill_countsが空でないことを確認してからDataFrameを作成
    if skill_counts:
        skill_df = pd.DataFrame(skill_counts.items(), columns=['Skill', 'Count']).sort_values('Count', ascending=False)
        sns.barplot(x='Count', y='Skill', data=skill_df.head(20)) # 上位20スキル
        plt.title('Top 20 Skills Found Across All CVs')
        plt.tight_layout()
        plt.show()
    else:
        print("No skill counts to display in the bar graph.")


    # 各人のスキル数を比較
    person_skill_counts = {person: len(skills) for person, skills in extracted_skills.items() if skills} # スキルがある人だけを対象
    if person_skill_counts: # スキルを持つ人がいる場合のみプロット
        plt.figure(figsize=(8, 5))
        sns.barplot(x=list(person_skill_counts.keys()), y=list(person_skill_counts.values()))
        plt.title('Number of Identified Skills per Person')
        plt.ylabel('Number of Skills')
        plt.show()
    else:
        print("No skills identified for any person to compare counts.")


    # 3者のスキル共通部分と独自部分 (簡易的な集合演算)
    if len(extracted_skills) == 3: # 3人全員のデータがある場合
        # extracted_skillsのキーの順序に依存しないように、明示的に名前でアクセスすることを推奨
        # ただし、ここでは元のコードのロジックを維持
        persons_names_list = list(extracted_skills.keys())
        if len(persons_names_list) ==3 : # キーが3つあることを確認
            skills_p1 = set(extracted_skills.get(persons_names_list[0], [])) # .getで見つからない場合のエラーを回避
            skills_p2 = set(extracted_skills.get(persons_names_list[1], []))
            skills_p3 = set(extracted_skills.get(persons_names_list[2], []))

            common_skills_all = skills_p1.intersection(skills_p2, skills_p3)
            print(f"\nSkills common to all three: {common_skills_all if common_skills_all else 'None'}")

            # Arale & Yanay の共通スキル
            common_AY = skills_p1.intersection(skills_p2) - common_skills_all
            print(f"Skills common to {persons_names_list[0]} & {persons_names_list[1]} (but not {persons_names_list[2]}): {common_AY if common_AY else 'None'}")

            # Yasuyuki 独自のスキル (Arale, Yanayが持っていないもの)
            unique_Y = skills_p3 - (skills_p1.union(skills_p2))
            print(f"Skills unique to {persons_names_list[2]} (compared to {persons_names_list[0]} & {persons_names_list[1]}): {unique_Y if unique_Y else 'None'}")
        else:
            print("Could not perform 3-way skill comparison as data for exactly 3 persons is not available in extracted_skills.")
    elif extracted_skills: # データはあるが3人分ではない場合
        print("\nSkill comparison for common/unique skills is designed for exactly 3 persons. Adjust logic if needed for different number of persons.")


print("\nステップ5完了: スキル抽出と可視化が試みられました。")

In [ ]:
# --- ステップ6: トピックモデリング (LDA) - インデント修正・デバッグ強化版 ---
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from wordcloud import WordCloud
import matplotlib.pyplot as plt
# nltk.corpus.stopwords はステップ4でインポートとダウンロードがされている想定
# from nltk.corpus import stopwords
# stop_words_english = set(stopwords.words('english')) # ステップ4で定義されているはず

# === 設定値 ===
NUM_TOPICS = 3  # 各人物から抽出したいトピックの数 (テキスト内容が少ない場合は減らす)
NUM_WORDS_PER_TOPIC = 7 # 各トピックを代表する単語の数
MIN_WORDS_FOR_LDA = NUM_TOPICS * NUM_WORDS_PER_TOPIC * 2 # LDAを実行するための最低限の単語数（目安）

# === 結果格納用の辞書 ===
topic_models = {}
topic_wordclouds = {}

# === processed_texts の取得 ===
# analysis_results 辞書と processed_texts がステップ4までに正しく準備されているか確認
if 'analysis_results' in globals() and isinstance(analysis_results, dict) and \
   'processed_texts' in analysis_results and isinstance(analysis_results['processed_texts'], dict):
    current_processed_texts = analysis_results['processed_texts']
    print("Successfully loaded 'processed_texts' from 'analysis_results'.")
elif 'processed_texts' in globals() and isinstance(processed_texts, dict):
    current_processed_texts = processed_texts
    print("Warning: 'processed_texts' found in global scope, but not in 'analysis_results'. Make sure data flow is intended.")
    if 'analysis_results' not in globals() or not isinstance(analysis_results, dict):
        analysis_results = {} # analysis_results がなければ初期化
    analysis_results['processed_texts'] = current_processed_texts
else:
    print("CRITICAL ERROR: 'processed_texts' is not defined or not a dictionary. \n"
          "Please ensure Step 4 (Text Preprocessing) has been run successfully and "
          "'processed_texts' dictionary is created and populated.")
    current_processed_texts = {}

# === stop_words_english の取得 ===
if 'stop_words_english' not in globals():
    print("CRITICAL ERROR: 'stop_words_english' is not defined. \n"
          "Please ensure Step 4 (Text Preprocessing) has been run successfully, including nltk stopword download and definition.")
    try:
        from nltk.corpus import stopwords
        stop_words_english = set(stopwords.words('english'))
        print("Fallback: Defined 'stop_words_english' in Step 6. Ideally, this should be done in Step 4.")
    except Exception as e_nltk:
        print(f"Fallback Error: Could not define 'stop_words_english': {e_nltk}")
        stop_words_english = set()
else:
    print(f"Successfully loaded 'stop_words_english' (contains {len(stop_words_english)} words).")


print("\n--- Starting Topic Modeling (LDA) ---")

if not current_processed_texts:
    print("No processed texts available to perform LDA. Skipping topic modeling.")
else:
    for person, text_content in current_processed_texts.items():
        print(f"\n--- Processing for: {person} ---")

        # 1. 入力テキストの検証とデバッグ出力
        print(f"  1. Verifying input text for {person}:")
        if not isinstance(text_content, str) or not text_content.strip():
            print(f"  !!! Issue: Processed text for {person} is empty, not a string, or whitespace only. Skipping LDA for this person.")
            print(f"     Content: '{text_content}'")
            continue

        print(f"     First 300 characters of processed text: '{text_content[:300]}...'")
        words_in_text = text_content.split()
        num_words = len(words_in_text)
        print(f"     Approximate word count: {num_words}")

        if num_words < MIN_WORDS_FOR_LDA:
            print(f"  !!! Issue: Text for {person} has only {num_words} words. "
                  f"This might be too short for meaningful LDA with {NUM_TOPICS} topics. "
                  f"(Minimum recommended: {MIN_WORDS_FOR_LDA} words). Skipping LDA for this person.")
            continue

        print(f"  2. Attempting CountVectorizer for {person}:")
        try:
            # === CountVectorizerのパラメータ設定 ===
            # まずは最も単純な設定で試す (stop_wordsのみ)
            # これでエラーが出る場合、stop_wordsリストか前処理自体に問題がある可能性が高い
            vectorizer = CountVectorizer(
                stop_words=list(stop_words_english)
            )

            # (次の試行案1：min_df, max_df, ngram_range を少し設定してみる)
            # vectorizer = CountVectorizer(
            #     stop_words=list(stop_words_english),
            #     min_df=2,       # 最低2回出現する単語
            #     max_df=0.95,    # 上位95%の頻出語は除外 (高頻度すぎる一般的単語対策)
            #     ngram_range=(1, 2) # 1単語と2単語の組み合わせ
            # )

            # (次の試行案2：ストップワードなしで試す - 原因切り分け用)
            # vectorizer = CountVectorizer(
            #     stop_words=None # ストップワード除去を一時的に無効化
            # )

            document_term_matrix = vectorizer.fit_transform([text_content])
            feature_names = vectorizer.get_feature_names_out()
            num_features = len(feature_names)

            print(f"     Number of features (unique words) found by CountVectorizer: {num_features}")
            if num_features == 0:
                print(f"  !!! Issue: CountVectorizer found 0 features for {person}. \n"
                      "     This means no words remained after applying current CountVectorizer settings.\n"
                      "     Suggestions:\n"
                      "       - If using `stop_words`, ensure it's not removing everything. Try `stop_words=None` temporarily.\n"
                      "       - If `min_df` is high or `max_df` is low, adjust them.\n"
                      "       - Review Step 4 (Preprocessing) to ensure text is not overly cleaned (e.g., regex removing too much).\n"
                      "     Skipping LDA for this person.")
                # (デバッグ用) どの単語が残るか確認
                # if text_content.strip(): # テキストが空でない場合のみ
                #    temp_vec_no_filter = CountVectorizer()
                #    temp_vec_no_filter.fit([text_content])
                #    print(f"     (Debug) Features without any CountVectorizer filters (except tokenization): {len(temp_vec_no_filter.get_feature_names_out())}")
                #    print(f"     (Debug) Sample unfiltered features: {temp_vec_no_filter.get_feature_names_out()[:20]}")
                continue

            print(f"     Sample features extracted by CountVectorizer: {feature_names[:min(20, num_features)]}")


            print(f"  3. Attempting LDA for {person} with {num_features} features:")

            current_num_topics_for_person = NUM_TOPICS
            if num_features < NUM_TOPICS:
                print(f"     Warning: Number of features ({num_features}) is less than NUM_TOPICS ({NUM_TOPICS}). "
                      f"Adjusting NUM_TOPICS to {num_features} for this person.")
                current_num_topics_for_person = num_features

            if current_num_topics_for_person == 0:
                 print(f"     Error: Cannot perform LDA as effective number of topics is 0 for {person}.")
                 continue

            lda_model = LatentDirichletAllocation(
                n_components=current_num_topics_for_person,
                random_state=42,
                learning_method='online',
            )
            # (次の試行案：LDAの事前分布パラメータ調整)
            # lda_model = LatentDirichletAllocation(
            #     n_components=current_num_topics_for_person,
            #     random_state=42,
            #     learning_method='online',
            #     doc_topic_prior=0.1,  # alpha
            #     topic_word_prior=0.01 # eta
            # )
            lda_model.fit(document_term_matrix)

            person_topic_list = []
            print(f"     Top topics found for {person}:")
            for topic_idx, topic_distribution in enumerate(lda_model.components_):
                if topic_distribution.sum() == 0:
                    print(f"     - Topic #{topic_idx + 1}: Empty distribution (no words).")
                    continue
                top_word_indices = topic_distribution.argsort()[:-NUM_WORDS_PER_TOPIC - 1:-1]
                top_words = [feature_names[i] for i in top_word_indices]
                person_topic_list.append(top_words)
                print(f"     - Topic #{topic_idx + 1}: {', '.join(top_words)}")

            topic_models[person] = person_topic_list

            print(f"  4. Generating Word Cloud for {person} topics:")
            all_topic_words_for_cloud = ' '.join([' '.join(words) for words in person_topic_list if words])
            if all_topic_words_for_cloud.strip():
                try:
                    wordcloud_generator = WordCloud(
                        width=800, height=400,
                        background_color='white',
                        stopwords=list(stop_words_english)
                    ).generate(all_topic_words_for_cloud)

                    topic_wordclouds[person] = wordcloud_generator
                    plt.figure(figsize=(10, 5))
                    plt.imshow(wordcloud_generator, interpolation='bilinear')
                    plt.axis('off')
                    plt.title(f'Topic Word Cloud for {person}')
                    plt.show()
                except ValueError as e_wc:
                    print(f"     !!! Issue: Could not generate word cloud for {person}. Error: {e_wc}")
                except Exception as e_wc_generic:
                    print(f"     !!! Issue: An unexpected error occurred during word cloud generation for {person}. Error: {e_wc_generic}")
            else:
                print(f"     Skipping word cloud for {person} as no valid topic words were found.")

        except ValueError as e_vec_lda:
            print(f"  !!! CRITICAL ERROR during LDA process for {person}: {e_vec_lda}\n"
                  "     This often means 'max_df corresponds to < documents than min_df' or similar issues "
                  "     related to vocabulary size or document length after vectorization.\n"
                  "     Suggestions in the error message above and in the code comments should be checked.")
        except Exception as e_generic_person:
            print(f"  !!! An unexpected CRITICAL ERROR occurred while processing {person}: {e_generic_person}")


# === 結果の格納 ===
if 'analysis_results' not in globals() or not isinstance(analysis_results, dict):
    analysis_results = {}
analysis_results['topic_models'] = topic_models

print("\n--- ステップ6完了: トピックモデリングが試みられました。 ---")
print("    詳細なデバッグ情報とエラーメッセージ（もしあれば）を上記で確認してください。")

In [ ]:
# 有効な処理済みテキストのリストと対応する人物名リストを作成
persons_for_similarity = [p for p, t in processed_texts.items() if t]
texts_for_similarity = [processed_texts[p] for p in persons_for_similarity if processed_texts[p]]

if len(texts_for_similarity) < 2:
    print("Not enough valid texts (need at least 2) to perform similarity analysis. Skipping.")
else:
    print("\nPerforming Text Similarity Analysis...")
    try:
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(texts_for_similarity)

        # コサイン類似度を計算
        cosine_sim_matrix = cosine_similarity(tfidf_matrix)

        # 結果をDataFrameで表示
        similarity_df = pd.DataFrame(cosine_sim_matrix, index=persons_for_similarity, columns=persons_for_similarity)
        analysis_results['similarity_matrix'] = similarity_df

        print("Cosine Similarity Matrix:")
        print(similarity_df)

        # ヒートマップで可視化
        plt.figure(figsize=(8, 6))
        sns.heatmap(similarity_df, annot=True, cmap="YlGnBu", fmt=".2f")
        plt.title('CV Text Similarity (Cosine Similarity on TF-IDF)')
        plt.show()

    except Exception as e:
        print(f"Error during similarity analysis: {e}")

print("\nステップ7完了: テキスト類似度分析が試みられました。")

In [ ]:
print("\n\n--- 統合分析レポート ---")
print("========================\n")

# 0. 基本情報
print("## 0. 分析対象")
for person in persons_files.keys():
    print(f"- {person}")
print("\n")

# 1. スキル分析サマリー
print("## 1. スキル分析概要")
if 'extracted_skills' in analysis_results:
    for person, skills in analysis_results['extracted_skills'].items():
        print(f"\n### {person}:")
        if skills:
            print(f"- 主な確認スキル ({len(skills)}個): {', '.join(skills[:15])}{'...' if len(skills) > 15 else ''}")
        else:
            print("- 確認された特定のスキルはありません")

    # スキル補完性に関するコメント (ステップ5の結果を基に)
    skills_data = analysis_results.get('extracted_skills', {})
    if len(skills_data) == 3: # 3人全員のデータがあることを想定
        p_names = list(skills_data.keys()) # 人物名のリストを取得
        # 人物名が 'Arale Cohen', 'Yanay Geva', 'Yasuyuki Sakane' の順であることを期待するが、
        # より堅牢にするには、persons_filesのキーの順序に依存しない方法で人物を特定する
        # ここでは、キーの順序に依存する形で実装（もし順序が異なる場合は要調整）
        name_arale = p_names[0]
        name_yanay = p_names[1]
        name_yasuyuki = p_names[2]

        s_arale = set(skills_data.get(name_arale, []))
        s_yanay = set(skills_data.get(name_yanay, []))
        s_yasuyuki = set(skills_data.get(name_yasuyuki, []))


        common_all = s_arale.intersection(s_yanay, s_yasuyuki)
        unique_yasuyuki = s_yasuyuki - (s_arale.union(s_yanay))

        print("\n### スキルの相互補完性 (概念的):")
        if common_all:
            print(f"- 3者共通スキル: {', '.join(list(common_all)[:5])}{'...' if len(common_all) > 5 else ''}")
        if unique_yasuyuki:
            print(f"- Yasuyuki Sakane氏のユニークスキル ({name_arale}氏, {name_yanay}氏との比較): {', '.join(list(unique_yasuyuki)[:5])}{'...' if len(unique_yasuyuki) > 5 else ''}")
            print("  - これらのユニークスキルは、チームの技術的専門性を拡張し、新しい事業領域の探索に貢献する可能性があります。")

        # Arale & Yanay が持ち、Yasuyukiが持たないスキル
        AY_only_skills = (s_arale.union(s_yanay)) - s_yasuyuki
        if AY_only_skills:
             print(f"- {name_arale}氏と{name_yanay}氏の ({name_yasuyuki}氏との比較) 主な保有スキル: {', '.join(list(AY_only_skills)[:5])}{'...' if len(AY_only_skills) > 5 else ''}")
             print("  - これらはVC運営および投資関連のコアコンピタンスである可能性が高く、Yasuyuki氏の技術専門性と結合することでシナジーが期待されます。")
else:
    print("スキル分析結果がありません。")
print("\n")


# 2. トピックモデリングサマリー
print("## 2. 主要トピック (専門分野) 概要")
if 'topic_models' in analysis_results and analysis_results['topic_models']:
    for person, topics in analysis_results['topic_models'].items():
        print(f"\n### {person}:")
        if topics:
            for i, topic_words in enumerate(topics):
                print(f"- トピック {i+1}: {', '.join(topic_words)}")
            print(f"  - {person}氏のCVは、主に上記のようなトピックに関連する経験/専門性を含んでいる可能性が高いです。")
        else:
            print("- 主要なトピックを抽出できませんでした。")
else:
    print("トピックモデリングの結果がありません。")
print("\n")

# 3. テキスト類似度サマリー
print("## 3. CV内容の類似度分析概要")
if 'similarity_matrix' in analysis_results:
    sim_matrix = analysis_results['similarity_matrix']
    print("以下は、各人物のCV内容間のコサイン類似度行列です (1に近いほど類似度が高いことを示します):")
    print(sim_matrix)

    persons_list = list(sim_matrix.columns)
    if len(persons_list) == 3: # 3人の場合
        # persons_listの順序が 'Arale Cohen', 'Yanay Geva', 'Yasuyuki Sakane' であると仮定
        # persons_filesのキーの順序と一致することを期待
        p1_name = persons_list[0]
        p2_name = persons_list[1]
        p3_name = persons_list[2]

        # AraleとYanayの類似度
        sim_AY = sim_matrix.loc[p1_name, p2_name]
        # AraleとYasuyukiの類似度
        sim_AYasu = sim_matrix.loc[p1_name, p3_name]
        # YanayとYasuyukiの類似度
        sim_YYasu = sim_matrix.loc[p2_name, p3_name]

        print(f"\n- {p1_name}氏と{p2_name}氏のCV類似度: {sim_AY:.2f}")
        print(f"- {p1_name}氏と{p3_name}氏のCV類似度: {sim_AYasu:.2f}")
        print(f"- {p2_name}氏と{p3_name}氏のCV類似度: {sim_YYasu:.2f}")

        if sim_AY > sim_AYasu and sim_AY > sim_YYasu:
            print(f"  - {p1_name}氏と{p2_name}氏は、{p3_name}氏に比べて相対的に類似した経験/経歴背景を持つ可能性があります (既存VC創業者としての共通点など)。")
        elif sim_AYasu > sim_AY and sim_YYasu > sim_AY : # 修正: Yasuyuki氏と他の2人のどちらかとの類似性が高い場合
             # この条件は少し曖昧なので、個別の比較でコメントする方が良いかもしれません
             print(f"  - {p3_name}氏は、{p1_name}氏または{p2_name}氏のどちらかと、一定の類似した経験/経歴背景を持つ可能性があります。")


        avg_sim_yasu_vs_founders = (sim_AYasu + sim_YYasu) / 2
        print(f"  - {p3_name}氏と既存創業者({p1_name}氏, {p2_name}氏)間の平均CV類似度: {avg_sim_yasu_vs_founders:.2f}")
        print("  - この類似度は、履歴書に記述された内容の言語的表現、プロジェクトの種類、使用された用語などの共通点を反映する可能性があります。")

else:
    print("テキスト類似度分析の結果がありません。")
print("\n")


# 4. 3者間の相関性、共起関係、シナジー予測 (概念的分析)
print("## 4. 3者間の相互作用、共創シナジー、成長加速に関する予測 (概念的分析)")
print("""
Yasuyuki Sakane氏の参画は、Arale Cohen氏とYanay Geva氏のVCファームに以下のような多面的なポジティブな影響をもたらすと予測されます。

### A. 技術的専門性の飛躍的向上とデューデリジェンス強化:
- Yasuyuki氏の深い技術的知見（例：AI、特定技術ドメイン）は、投資候補企業の技術デューデリジェンスの精度と深度を大幅に向上させます。
- 特に、日本市場における高度な技術を持つスタートアップの発掘、評価、および技術的課題の特定において、比類なき能力を発揮します。
- 投資後のポートフォリオ企業に対し、具体的な技術アドバイスやアーキテクチャレビュー、開発戦略支援などを提供し、企業価値向上に直接的に貢献します。

### B. 事業拡大と戦略的オプションの多様化:
- **日本市場への本格参入加速:** Yasuyuki氏の持つ日本市場や技術コミュニティへの深い理解とネットワークは、日本市場への参入戦略を具体化し、実行を加速させる上で不可欠な要素となります。現地パートナーシップ構築、ローカライズ戦略、日本特有のビジネス慣行への対応などが円滑に進むことが期待されます。
- **新規技術領域への投資拡大:** Yasuyuki氏の専門分野（例：AI, ブロックチェーン等）への知見を活用し、VCファームとして新たな成長領域への投資を戦略的に拡大できます。これは、ファームの専門性を高め、競争優位性を確立する上で重要です。
- **イノベーションの促進:** Arale氏とYanay氏の持つ広範なビジネス経験や投資実績と、Yasuyuki氏の最先端技術への洞察が融合することで、従来の発想にとらわれない革新的なビジネスモデルや投資テーマの創出が期待されます。

### C. Arale Cohen氏とYanay Geva氏の関係性の変化・変革・成長:
- **意思決定プロセスの質の向上:** Yasuyuki氏からの技術的視点に基づく客観的なインプットは、Arale氏とYanay氏の議論をより多角的で深いものにし、最終的な投資判断や戦略決定の質を向上させます。特に技術リスクの評価において重要な役割を果たします。
- **役割分担の最適化と効率化:** Yasuyuki氏が技術関連の評価や目利き、ポートフォリオ企業の技術支援といった専門領域を担当することで、Arale氏とYanay氏は、ファンドレイジング、LPリレーション、大規模な事業戦略、ネットワーキングといった、それぞれの強みを活かせる領域により一層注力できるようになります。
- **相互学習と視点の拡張による成長:** Yasuyuki氏の異なるバックグラウンド、専門知識、そして日本という異なる文化圏からの視点は、Arale氏とYanay氏に新たな気づきや学びをもたらし、両者の視野を広げ、リーダーとしての成長を促進します。これにより、チーム全体の適応力と問題解決能力が向上します。
- **触媒としての役割:** Yasuyuki氏の参画は、時にArale氏とYanay氏の間で生じる可能性のある意見の相違や膠着状態に対し、新たな視点や解決策を提示する「触媒」としての役割を果たし、より建設的な議論と強固なパートナーシップへと導く可能性があります。

### D. 複利的な成長とビジネススケール拡大のステップ（予測）:
1.  **フェーズ1: 基盤構築と初期シナジー（〜1年）**
    *   Yasuyuki氏による技術評価プロセスの導入・標準化。既存・新規ディールの技術DDへの全面的な関与。
    *   日本市場における初期的なネットワーク構築と情報収集、有望な技術シーズの特定。
    *   VCファーム内での技術トレンド勉強会などを通じた知識共有とチーム全体の技術リテラシー向上。
    *   Arale氏・Yanay氏との信頼関係構築と、3者の役割・責任範囲の明確化。
2.  **フェーズ2: 戦略実行と成果の可視化（1〜3年）**
    *   日本市場における具体的な投資案件の実行、または日本企業との戦略的提携の実現。
    *   Yasuyuki氏の専門性を活かした特定技術分野特化型の小規模ファンド（または投資枠）の検討・立ち上げ。
    *   ポートフォリオ企業への技術支援による成功事例の創出（例：製品開発サイクルの短縮、技術的課題の解決）。
    *   3者の連携によるディールソーシングチャネルの拡大と質の向上。
3.  **フェーズ3: スケール拡大と持続的成長（3年〜）**
    *   日本市場での成功を足がかりとしたアジア他地域への展開可能性の検討。
    *   VCファームが「技術に強い投資ファーム」としての確固たるブランドを確立。
    *   Yasuyuki氏がリーダーシップを発揮し、技術チームの育成や次世代の専門家採用を推進。
    *   継続的なイノベーションと相互補完による複利的な成長モデルを確立し、ファンドパフォーマンスの最大化とVCファームの持続的なスケール拡大を実現。

### E. 成功のための重要因子:
- **明確なビジョン共有:** 3者がVCファームの将来像とYasuyuki氏の役割について共通認識を持つこと。
- **オープンなコミュニケーション:** 定期的な情報共有、率直な意見交換、建設的なフィードバックが不可欠。
- **文化の融合と相互尊重:** 異なるバックグラウンドを持つメンバー間の文化的な違いを理解し、尊重し合う組織文化の醸成。
- **柔軟な役割分担と権限委譲:** 状況の変化に応じて役割を柔軟に見直し、Yasuyuki氏の専門性を最大限に活かせるような適切な権限委譲。
- **成果の評価とインセンティブ:** Yasuyuki氏の貢献を適切に評価し、モチベーションを維持するためのインセンティブ設計。
""")

print("========================")
print("--- 分析レポート終了 ---")

In [ ]:
# --- ステップ7: 共起関係分析 (Co-occurrence Network) ---
import itertools
from collections import Counter
import networkx as nx # networkx をインポート
# matplotlib.pyplot は既にインポートされているはず (pltとして)

# ステップ7の冒頭、tokens を作成する前に追加
# stop_words_english はステップ6などで定義・更新されている想定

#text_for_network = current_processed_texts[target_person_for_network]
#raw_tokens = text_for_network.split()
#tokens = [token for token in raw_tokens if token not in stop_words_english and len(token) > 1] # ストップワード除去と短い単語除去

print("\n--- ステップ7: 共起関係分析 ---")

# 分析対象の人物を選択 (ここでは例として Yanay Geva)
# 全員分実行する場合はループ処理にする
target_person_for_network = 'Yasuyuki Sakane' # または他の人物名

if target_person_for_network in current_processed_texts and current_processed_texts[target_person_for_network]:
    text_for_network = current_processed_texts[target_person_for_network]
    tokens = text_for_network.split() # 簡単のためスペースで単語分割

    # 共起をカウントするためのウィンドウサイズ (例: 同じ文脈と見なす単語の距離)
    window_size = 5
    # 最低共起回数 (これ以下のペアはネットワークに表示しない)
    min_cooccurrence = 10 # テキスト量や見たい粒度で調整

    co_occurrences = Counter()

    # ウィンドウをスライドさせながら共起ペアをカウント
    for i in range(len(tokens) - window_size + 1):
        window = sorted(list(set(tokens[i : i + window_size]))) # ウィンドウ内のユニークな単語 (ソートしてペアの順序を固定)
        for w1, w2 in itertools.combinations(window, 2): # ウィンドウ内の全ての単語ペア
            if w1 != w2: # 同じ単語同士のペアは除く (itertools.combinationsなら不要だが念のため)
                # 単語のペアをアルファベット順にしてカウント (例: (apple, banana) と (banana, apple) を同じと見なす)
                pair = tuple(sorted((w1, w2)))
                co_occurrences[pair] += 1

    # 共起回数がmin_cooccurrence以上のペアのみを抽出
    significant_co_occurrences = {pair: count for pair, count in co_occurrences.items() if count >= min_cooccurrence}

    if not significant_co_occurrences:
        print(f"No significant co-occurrences found for {target_person_for_network} with min_cooccurrence={min_cooccurrence}.")
    else:
        print(f"Found {len(significant_co_occurrences)} significant co-occurrence pairs for {target_person_for_network}.")

        # NetworkXグラフの作成
        G = nx.Graph()
        for pair, weight in significant_co_occurrences.items():
            G.add_edge(pair[0], pair[1], weight=weight)

        if not G.nodes():
            print(f"Graph for {target_person_for_network} has no nodes. Cannot draw.")
        else:
            plt.figure(figsize=(25, 25))
            pos = nx.spring_layout(G, k=0.5, iterations=50, seed=42) # ノードの配置アルゴリズム

            # ノードのサイズを次数（接続しているエッジの数）に応じて変更
            node_sizes = [G.degree(node) * 50 for node in G.nodes()]

            # エッジの太さを共起回数（重み）に応じて変更
            edge_widths = [G.edges[edge]['weight'] * 0.5 for edge in G.edges()]

            nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color='skyblue', alpha=0.8)
            nx.draw_networkx_edges(G, pos, width=edge_widths, edge_color='gray', alpha=0.5)
            nx.draw_networkx_labels(G, pos, font_size=8)

            plt.title(f'Co-occurrence Network for {target_person_for_network} (min_cooccurrence={min_cooccurrence})', size=15)
            plt.axis('off')
            plt.show()

            # (オプション) 特に中心性の高い単語を表示
            # try:
            #     degree_centrality = nx.degree_centrality(G)
            #     sorted_degree = sorted(degree_centrality.items(), key=lambda item: item[1], reverse=True)
            #     print(f"\nTop 10 words by Degree Centrality for {target_person_for_network}:")
            #     for i, (word, centrality) in enumerate(sorted_degree[:10]):
            #         print(f"{i+1}. {word}: {centrality:.3f}")
            # except Exception as e_centrality:
            #     print(f"Could not calculate centrality: {e_centrality}")

else:
    print(f"No processed text found for {target_person_for_network} to perform co-occurrence analysis.")

print("\n--- ステップ7完了: 共起関係分析が試みられました。 ---")